# Handling Email (sending / receiving)

In [1]:
import smtplib

Most of the time, you'll be connecting to either your ISP's SMTP server, or some other server with authentication. In order to avoid exposing my password and such, I'll just use a local proxy server to print some debugging information:

(You can run the proxy with `python -m smtpd -n -c DebuggingServer`)

In [2]:
conn = smtplib.SMTP('localhost', 8025)
conn.set_debuglevel(1)
# conn.login(username, password)
addr_from = 'rick@arborian.com'
addr_to = 'rick446@usa.net'
msg = '''Subject: This is a test
From: <{}>
To: <{}>

Test message
'''.format(addr_from, addr_to)
conn.sendmail(addr_from, [addr_to], msg)

send: 'ehlo faramir.local\r\n'
reply: b'250-faramir.local\r\n'
reply: b'250-8BITMIME\r\n'
reply: b'250 HELP\r\n'
reply: retcode (250); Msg: b'faramir.local\n8BITMIME\nHELP'
send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Subject: This is a test\r\nFrom: <rick@arborian.com>\r\nTo: <rick446@usa.net>\r\n\r\nTest message\r\n.\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
data: (250, b'OK')


{}

## Creating and parsing email messages

If you're going to be putting together more complicated emails, the `email` module is extremely useful (for multipart messages, attachments, etc.):

In [3]:
from email.mime.text import MIMEText

In [4]:
msg = MIMEText('This is a text message')
msg['Subject'] = 'This is a test message'
msg['From'] = addr_from
msg['To'] = addr_to
conn.sendmail(addr_from, [addr_to], msg.as_string())

send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Content-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\nSubject: This is a test message\r\nFrom: rick@arborian.com\r\nTo: rick446@usa.net\r\n\r\nThis is a text message\r\n.\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
data: (250, b'OK')


{}

In [5]:
from email.mime.multipart import MIMEMultipart

In [6]:
msg = MIMEMultipart()
msg['Subject'] = 'This is a test message'
msg['From'] = addr_from
msg['To'] = addr_to
text_part = MIMEText('This is a text message', 'plain')
html_part = MIMEText('<h1>This is an HTML message</h1>', 'html')
msg.attach(text_part)
msg.attach(html_part)
conn.sendmail(addr_from, [addr_to], msg.as_string())

send: 'mail FROM:<rick@arborian.com>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'rcpt TO:<rick446@usa.net>\r\n'
reply: b'250 OK\r\n'
reply: retcode (250); Msg: b'OK'
send: 'data\r\n'
reply: b'354 End data with <CR><LF>.<CR><LF>\r\n'
reply: retcode (354); Msg: b'End data with <CR><LF>.<CR><LF>'
data: (354, b'End data with <CR><LF>.<CR><LF>')
send: b'Content-Type: multipart/mixed; boundary="===============7485811049391948111=="\r\nMIME-Version: 1.0\r\nSubject: This is a test message\r\nFrom: rick@arborian.com\r\nTo: rick446@usa.net\r\n\r\n--===============7485811049391948111==\r\nContent-Type: text/plain; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\n\r\nThis is a text message\r\n--===============7485811049391948111==\r\nContent-Type: text/html; charset="us-ascii"\r\nMIME-Version: 1.0\r\nContent-Transfer-Encoding: 7bit\r\n\r\n<h1>This is an HTML message</h1>\r\n--===============7485811049391948111==--\r\n.\r\n'
reply: b'250 OK\r\n'
re

{}

In [7]:
from email.parser import Parser

In [8]:
parsed = Parser().parsestr(msg.as_string())

In [9]:
parsed

In [10]:
headers = dict(parsed)
headers

{'Content-Type': 'multipart/mixed; boundary="===============7485811049391948111=="',
 'MIME-Version': '1.0',
 'Subject': 'This is a test message',
 'From': 'rick@arborian.com',
 'To': 'rick446@usa.net'}

In [11]:
for part in parsed.walk():
    if part.is_multipart():
        continue
    print('>>> Begin part (class = {}) <<<'.format(part.__class__))
    print(dict(part))
    print(part)
    print('>>> End part <<<')
    print('')

>>> Begin part (class = <class 'email.message.Message'>) <<<
{'Content-Type': 'text/plain; charset="us-ascii"', 'MIME-Version': '1.0', 'Content-Transfer-Encoding': '7bit'}
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

This is a text message
>>> End part <<<

>>> Begin part (class = <class 'email.message.Message'>) <<<
{'Content-Type': 'text/html; charset="us-ascii"', 'MIME-Version': '1.0', 'Content-Transfer-Encoding': '7bit'}
Content-Type: text/html; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

<h1>This is an HTML message</h1>
>>> End part <<<



## Accessing email on the server using `imaplib`

In [12]:
import imaplib
server = imaplib.IMAP4_SSL('imap.gmail.com', 993)
server.login('class@arborian.com', 'class-test1')

('OK', [b'class@arborian.com authenticated (Success)'])

In [13]:
server.select()
typ, data = server.search(None, 'ALL')
for num in data[0].split():
    typ, data = server.fetch(num, '(RFC822)')
    msg = Parser().parsestr(str(data[0][1], 'utf-8'))
    print('Message {}\n{}\n'.format(num, dict(msg)))

Message b'1'
{'MIME-Version': '1.0', 'x-no-auto-attachment': '1', 'Received': 'by 2002:a2e:5015:0:0:0:0:0; Wed, 13 Mar 2019 13:28:31 -0700 (PDT)', 'Date': 'Wed, 13 Mar 2019 13:28:31 -0700', 'Message-ID': '<CAOHibm=bSyF3xSu863YLBO1pL04fGQQ2p2OkZnXJEbNf=0ww8Q@mail.gmail.com>', 'Subject': 'The best of Gmail, wherever you are', 'From': 'Gmail Team <mail-noreply@google.com>', 'To': 'Class Test2 <class@arborian.com>', 'Content-Type': 'multipart/alternative; boundary="0000000000003588bb0583ffa67d"'}

Message b'2'
{'MIME-Version': '1.0', 'x-no-auto-attachment': '1', 'Received': 'by 2002:a2e:5015:0:0:0:0:0; Wed, 13 Mar 2019 13:28:32 -0700 (PDT)', 'Date': 'Wed, 13 Mar 2019 13:28:32 -0700', 'Message-ID': '<CAOHibmkUys2rsKVHiy5WZRUAND7OcqWgVy1sCdj=SyPh20MuxQ@mail.gmail.com>', 'Subject': 'Tips for using your new inbox', 'From': 'Gmail Team <mail-noreply@google.com>', 'To': 'Class Test2 <class@arborian.com>', 'Content-Type': 'multipart/alternative; boundary="0000000000004069740583ffa680"'}



In [14]:
server.close()
server.logout()

('BYE', [b'LOGOUT Requested'])

# Lab

Open [Email Lab](./email-lab.ipynb)